In [1]:
import findspark
findspark.init('/home/gani/spark-2.4.4-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

In [3]:
spark = SparkSession.builder.appName('ch4ExplainFunction').getOrCreate()

In [4]:
df = spark.read.csv('NCHS_-_Teen_Birth_Rates_for_Age_Group_15-19_in_the_United_States_by_County.csv',
                   inferSchema=True,header=True)

In [5]:
df0 = df

In [6]:
df = df.union(df0)

In [10]:
df = df.withColumnRenamed('Lower Confidence Limit','lcl') \
        .withColumnRenamed('Upper Confidence Limit','ucl')

In [11]:
df.show()

+----+-------+-------+---------------+----------------+------------------+----------+---------+---------+
|Year|  State| County|State FIPS Code|County FIPS Code|Combined FIPS Code|Birth Rate|      lcl|      ucl|
+----+-------+-------+---------------+----------------+------------------+----------+---------+---------+
|2003|Alabama|Autauga|              1|               1|              1001| 46.377215|40.683107|52.508481|
|2004|Alabama|Autauga|              1|               1|              1001| 46.050618|41.084735|51.340795|
|2005|Alabama|Autauga|              1|               1|              1001| 43.941062|39.513897|48.646647|
|2006|Alabama|Autauga|              1|               1|              1001| 43.826654|39.570766|48.345353|
|2007|Alabama|Autauga|              1|               1|              1001| 43.757806|39.573448|48.199268|
|2008|Alabama|Autauga|              1|               1|              1001| 42.169739|38.130402|46.458997|
|2009|Alabama|Autauga|              1|        

In [12]:
df = df.withColumn('avg', expr('(lcl+ucl)/2')) \
        .withColumn('lcl2', col('lcl')) \
        .withColumn('ucl2',col('ucl'))

In [13]:
df.show()

+----+-------+-------+---------------+----------------+------------------+----------+---------+---------+------------------+---------+---------+
|Year|  State| County|State FIPS Code|County FIPS Code|Combined FIPS Code|Birth Rate|      lcl|      ucl|               avg|     lcl2|     ucl2|
+----+-------+-------+---------------+----------------+------------------+----------+---------+---------+------------------+---------+---------+
|2003|Alabama|Autauga|              1|               1|              1001| 46.377215|40.683107|52.508481|         46.595794|40.683107|52.508481|
|2004|Alabama|Autauga|              1|               1|              1001| 46.050618|41.084735|51.340795|46.212765000000005|41.084735|51.340795|
|2005|Alabama|Autauga|              1|               1|              1001| 43.941062|39.513897|48.646647|         44.080272|39.513897|48.646647|
|2006|Alabama|Autauga|              1|               1|              1001| 43.826654|39.570766|48.345353|43.958059500000005|39.570

In [14]:
df.head(1)

[Row(Year=2003, State='Alabama', County='Autauga', State FIPS Code=1, County FIPS Code=1, Combined FIPS Code=1001, Birth Rate=46.377215, lcl=40.683107, ucl=52.508481, avg=46.595794, lcl2=40.683107, ucl2=52.508481)]

In [15]:
df.explain()

== Physical Plan ==
Union
:- *(1) Project [Year#10, State#11, County#12, State FIPS Code#13, County FIPS Code#14, Combined FIPS Code#15, Birth Rate#16, Lower Confidence Limit#17 AS lcl#147, Upper Confidence Limit#18 AS ucl#157, ((Lower Confidence Limit#17 + Upper Confidence Limit#18) / 2.0) AS avg#221, Lower Confidence Limit#17 AS lcl2#232, Upper Confidence Limit#18 AS ucl2#244]
:  +- *(1) FileScan csv [Year#10,State#11,County#12,State FIPS Code#13,County FIPS Code#14,Combined FIPS Code#15,Birth Rate#16,Lower Confidence Limit#17,Upper Confidence Limit#18] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/gani/GitRepos/sparkInAction/Ch4_FundamentallyLazy/NCHS_-_Teen_Birth_..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Year:int,State:string,County:string,State FIPS Code:int,County FIPS Code:int,Combined FIPS...
+- *(2) Project [Year#10, State#11, County#12, State FIPS Code#13, County FIPS Code#14, Combined FIPS Code#15, Birth Rate#16, Lower Confidence 

In [16]:
spark.stop()